# Cadquery + ipywidget + dotbimpy

In [1]:
from dotbimpy import *
from ipywidgets import interact
import cadquery
import uuid

In [2]:
def cadquery_mesh_to_dotbim_mesh(cadquery_mesh, mesh_id):
    vertices, triangles = cadquery_mesh
    coordinates = []
    for i in vertices:
        coordinates.extend([i.x, i.y, i.z])
    indices = [item for sublist in triangles for item in sublist]
    return Mesh(mesh_id=mesh_id, coordinates=coordinates, indices=indices)

In [3]:
def create_mesh_tube(plane, face, l, h, t, mesh_id):
    workplane = cadquery.Workplane(plane).circle(h).extrude(l).faces(face).workplane().circle(h-t).cutThruAll()
    mesh_cq = workplane.val().tessellate(0.1)
    mesh = cadquery_mesh_to_dotbim_mesh(mesh_cq, mesh_id)
    return mesh

def create_tube_element(vector, rotation, color, mesh_id, type_name):
    return Element(mesh_id=mesh_id,vector=vector,guid=str(uuid.uuid4()),
                   info={"Material": "Steel"},rotation=rotation,type=type_name,color=color)

In [10]:
def f(length):
    meshes = []
    elements = []
    
    # Creating chords
    meshes.append(create_mesh_tube('YZ', '>X', length, 0.03, 0.01, 0))
    elements.append(create_tube_element(Vector(0,0.25,0.25), Rotation(0,0,0,1), Color(0,0,255,255), 0, "Top Chord"))
    elements.append(create_tube_element(Vector(0,-0.25,0.25), Rotation(0,0,0,1), Color(0,0,255,255), 0, "Top Chord"))
    elements.append(create_tube_element(Vector(0,0.25,-0.25), Rotation(0,0,0,1), Color(0,0,255,255), 0, "Bottom Chord"))
    elements.append(create_tube_element(Vector(0,-0.25,-0.25), Rotation(0,0,0,1), Color(0,0,255,255), 0, "Bottom Chord"))
    
    # Creating posts
    meshes.append(create_mesh_tube('XY', '>Z', 0.46, 0.02, 0.005, 1))
    elements.append(create_tube_element(Vector(0.25,0.25,-0.23), Rotation(0,0,0,1), Color(0,255,0,255), 1, "Post"))
    elements.append(create_tube_element(Vector(0.25,-0.25,-0.23), Rotation(0,0,0,1), Color(0,255,0,255), 1, "Post"))
    elements.append(create_tube_element(Vector(length-0.25,0.25,-0.23), Rotation(0,0,0,1), Color(0,255,0,255), 1, "Post"))
    elements.append(create_tube_element(Vector(length-0.25,-0.25,-0.23), Rotation(0,0,0,1), Color(0,255,0,255), 1, "Post"))
    
    # Creating girder
    meshes.append(create_mesh_tube('XZ', '>Y', 0.46, 0.02, 0.005, 2))
    elements.append(create_tube_element(Vector(0.25,0.23,0.25), Rotation(0,0,0,1), Color(222,49,99,255), 2, "Girder"))
    elements.append(create_tube_element(Vector(0.25,0.23,-0.25), Rotation(0,0,0,1), Color(222,49,99,255), 2, "Girder"))
    elements.append(create_tube_element(Vector(length-0.25,0.23,0.25), Rotation(0,0,0,1), Color(222,49,99,255), 2, "Girder"))
    elements.append(create_tube_element(Vector(length-0.25,0.23,-0.25), Rotation(0,0,0,1), Color(222,49,99,255), 2, "Girder"))
    
    # Creating webs
    length_to_fill = length - 0.5
    number_of_gaps = int(length_to_fill / 0.5)
    meshes.append(create_mesh_tube('YZ', '>X', 1.41421*0.5, 0.015, 0.005, 3))
    for i in range(number_of_gaps):
        if i % 2 == 0:
            elements.append(create_tube_element(Vector(i*0.5+0.25,0.25,0.5/2.0), Rotation(0,0.382683,0,0.92388), Color(255,165,0,255), 3, "Web"))
            elements.append(create_tube_element(Vector(i*0.5+0.25,-0.25,-0.5/2.0), Rotation(0,-0.382683,0,0.92388), Color(255,165,0,255), 3, "Web"))
            elements.append(create_tube_element(Vector(i*0.5+0.25,0.25,0.5/2.0), Rotation(0,0,-0.382683,0.92388), Color(255,165,0,255), 3, "Web"))
            elements.append(create_tube_element(Vector(i*0.5+0.25,-0.25,-0.5/2.0), Rotation(0,0,0.382683,0.92388), Color(255,165,0,255), 3, "Web"))
        else:
            elements.append(create_tube_element(Vector(i*0.5+0.25,0.25,-0.5/2.0), Rotation(0,-0.382683,0,0.92388), Color(255,165,0,255), 3, "Web"))
            elements.append(create_tube_element(Vector(i*0.5+0.25,-0.25,0.5/2.0), Rotation(0,0.382683,0,0.92388), Color(255,165,0,255), 3, "Web"))
            elements.append(create_tube_element(Vector(i*0.5+0.25,-0.25,0.5/2.0), Rotation(0,0,0.382683,0.92388), Color(255,165,0,255), 3, "Web"))
            elements.append(create_tube_element(Vector(i*0.5+0.25,0.25,-0.5/2.0), Rotation(0,0,-0.382683,0.92388), Color(255,165,0,255), 3, "Web"))
    
    # Creating webs on ends
    meshes.append(create_mesh_tube('XZ', '>Y', 1.41421*0.5, 0.015, 0.005, 4))
    elements.append(create_tube_element(Vector(0.25,0.25,0.5/2.0), Rotation(0.382683,0,0,0.92388), Color(255,105,180,255), 4, "Web"))
    elements.append(create_tube_element(Vector(length-0.25,0.25,-0.5/2.0), Rotation(-0.382683,0,0,0.92388), Color(255,105,180,255), 4, "Web"))
    
    file = File("1.0.0", meshes=meshes, elements=elements, info={"Author": "John Doe"})
    file.view()
    file.save("Truss.bim")

In [12]:
interact(f, length=(2.0, 10.0, 1.00))

interactive(children=(FloatSlider(value=6.0, description='length', max=10.0, min=2.0, step=1.0), Output()), _d…

<function __main__.f(length)>